# 演習課題その1 ベクトル空間モデル

## 必須課題（1） 与えられたコーパスに対する検索の実現

h30iro/data/kyoto_results_100.jsonのデータを読み込みそれに対する検索を実現した。また検索結果としてクエリとの類似度がもっとも高い5つの文章のsummaryを出力する関数を実装した。なおこの検索ではストップワードの除去は行なっていない。検索クエリとして{"嵐山"}、{"金閣寺", "銀閣寺"}、{"京都", "土産", "食事"}の3つを用意し実際に検索を行なった。

### 実装したプログラム
以下のプログラムを実装した。

In [150]:
# 文書の読み込み
# 1.のコーパスはjson形式で保管されている．
import json
with open("../data/kyoto_results_100.json", "r") as f:
    docs = json.load(f)
    
# 特徴ベクトルの生成
# stopwordは除去していない
import gensim
corpus = [d["bow"].lower().split() for d in docs] 
dictionary = gensim.corpora.Dictionary(corpus) #コーパスを与えて，単語->IDの辞書を作成する
id_corpus = [dictionary.doc2bow(document) for document in corpus]
tf_vectors = gensim.matutils.corpus2dense(id_corpus, len(dictionary)).T
tfidf_model = gensim.models.TfidfModel(id_corpus, normalize=False)
tfidf_corpus = tfidf_model[id_corpus] #id_corpusをtfidfで重み付けされたものに変換
tfidf_vectors = gensim.matutils.corpus2dense(tfidf_corpus, len(dictionary)).T

# コサイン類似度を計算する関数を用意
from scipy.spatial.distance import cosine
def cosine_sim(v1, v2):
    return 1.0 - cosine(v1, v2)

# クエリを受け取ってランキング結果を返す関数
# param query : クエリに含まれる単語の集合
def ranking(query):
    tfidf_q = tfidf_model[dictionary.doc2bow(query)]  #クエリをtfidfベクトルに変換
    query_vector = gensim.matutils.corpus2dense([tfidf_q], len(dictionary)).T[0]
    result = list(map(lambda i: (cosine_sim(query_vector, tfidf_vectors[i]), i), range(len(docs))))
    result.sort()
    result.reverse()
    return result

# クエリを受け取ってランキング結果上位5位のsummaryを返す関数
# param query : クエリに含まれる単語の集合
def getTop5Summary(query):
    result = ranking(query)
    return list(map(lambda i: docs[result[i][1]]["summary"], range(5)))

### 検索結果
検索クエリとして{"嵐山"}、{"金閣寺", "銀閣寺"}、{"京都", "土産", "食事"}の3つを用意し実際に検索を行なった。

In [152]:
getTop5Summary({"嵐山"})

['京都嵐山観光おすすめ！人気散策コースから周辺グルメまでご紹介！観光 京都の代表的な観光地で桜や紅葉の名所にもなっている嵐山。一日過ごしても飽きないくらい観たり食べたり買い物したりする人気スポットがいっぱいです。',
 '嵐山といえば、京都においても屈指のおすすめ観光スポットですね。今回は嵐山に注目してこれだけは外せない！な、人気の名所や定番の観光スポットをランキング形式でご案内します！デートでも家族旅行でも間違いなしの ...',
 '春は桜、秋は紅葉が全山を覆う嵐山。この嵐山を起点として嵯峨野の小径を散策するコースは、歴史や文学に関わる史跡が豊富にあります。家族旅行からカップル、一人旅まで、それぞれに楽しめる幅と奥行きがある散策コースです。',
 '見所の多い京都観光の中でも、特に人気が高い嵐山・嵯峨野エリア。この辺りは、古くから貴族や文人たちが都を逃れ「わび住まい」をした風流の地だけあり、周囲の山々が紅葉に染まる晩秋ともなれば、その風情もひとしおです。',
 '京都観光を楽しむ まとめ情報。京都の世界遺産や名所を紹介しています。 ... 京都観光 紅葉 京都紅葉の名称 京都紅葉おすすめ鑑賞スポット 東山や嵐山など、京都の紅葉は名所が広範囲にたくさんあるので']

この検索の結果は以下のようになった。

1. 京都嵐山観光おすすめ！人気散策コースから周辺グルメまでご紹介！観光 京都の代表的な観光地で桜や紅葉の名所にもなっている嵐山。一日過ごしても飽きないくらい観たり食べたり買い物したりする人気スポットがいっぱいです。
2. 嵐山といえば、京都においても屈指のおすすめ観光スポットですね。今回は嵐山に注目してこれだけは外せない！な、人気の名所や定番の観光スポットをランキング形式でご案内します！デートでも家族旅行でも間違いなしの ...
3. 春は桜、秋は紅葉が全山を覆う嵐山。この嵐山を起点として嵯峨野の小径を散策するコースは、歴史や文学に関わる史跡が豊富にあります。家族旅行からカップル、一人旅まで、それぞれに楽しめる幅と奥行きがある散策コースです。
4. 見所の多い京都観光の中でも、特に人気が高い嵐山・嵯峨野エリア。この辺りは、古くから貴族や文人たちが都を逃れ「わび住まい」をした風流の地だけあり、周囲の山々が紅葉に染まる晩秋ともなれば、その風情もひとしおです。
5. 京都観光を楽しむ まとめ情報。京都の世界遺産や名所を紹介しています。 ... 京都観光 紅葉 京都紅葉の名称 京都紅葉おすすめ鑑賞スポット 東山や嵐山など、京都の紅葉は名所が広範囲にたくさんあるので

In [156]:
getTop5Summary({"金閣寺", "銀閣寺"})

['金閣寺は京都を代表する観光地のひとつ。銀閣寺・嵐山・清水寺などとならび、五本の指に入る人気スポットです。ところが京都観光は移動が大変。交通手段も複雑です。そこで今回は、金閣寺から各観光名所・人気スポットへの ...',
 '京都観光の楽しみ方 京都観光といえば、金閣寺・銀閣寺の室町時代の将軍が建てた寺や 清水寺、高台寺、八坂神社などの東山エリアのお寺めぐりを楽しんだり 春には鮮やかな桜の花見、秋には美しい紅葉を鑑賞する京都観光や',
 '春夏の京都観光スポットを巡るおすすめバスツアー予約。バスガイドが京都のお寺や名所を案内する定期観光バスで行くJTBの日帰りバスツアー！比叡山や大原三千院、金閣寺、銀閣寺など京都観光に迷ったらこちのページをチェック。',
 '長きにわたって日本の首都として栄えた古都京都は、市街地に清水寺や金閣寺（鹿苑寺）、平安神宮や北野天満宮といった全国に名',
 '京都の観光名所は広範囲に散らばっているため、慣れていないと移動だけでひと苦労。限られた時間の中で効率よく見て回りたいなら、定期観光バスや観光タクシーが便利です。清水寺や金閣寺など人気の観光名所を巡る王道コースを ...']

この検索の結果は以下のようになった。

1. 金閣寺は京都を代表する観光地のひとつ。銀閣寺・嵐山・清水寺などとならび、五本の指に入る人気スポットです。ところが京都観光は移動が大変。交通手段も複雑です。そこで今回は、金閣寺から各観光名所・人気スポットへの ...
2. 京都観光の楽しみ方 京都観光といえば、金閣寺・銀閣寺の室町時代の将軍が建てた寺や 清水寺、高台寺、八坂神社などの東山エリアのお寺めぐりを楽しんだり 春には鮮やかな桜の花見、秋には美しい紅葉を鑑賞する京都観光や
3. 春夏の京都観光スポットを巡るおすすめバスツアー予約。バスガイドが京都のお寺や名所を案内する定期観光バスで行くJTBの日帰りバスツアー！比叡山や大原三千院、金閣寺、銀閣寺など京都観光に迷ったらこちのページをチェック。
4. 長きにわたって日本の首都として栄えた古都京都は、市街地に清水寺や金閣寺（鹿苑寺）、平安神宮や北野天満宮といった全国に名
5. 京都の観光名所は広範囲に散らばっているため、慣れていないと移動だけでひと苦労。限られた時間の中で効率よく見て回りたいなら、定期観光バスや観光タクシーが便利です。清水寺や金閣寺など人気の観光名所を巡る王道コースを ...

In [155]:
getTop5Summary({"京都", "土産", "食事"})

['JR東海の京都観光情報【そうだ 京都、行こう。公式サイト】。京都の風景、お寺や神社、イベント、お祭り、お食事やお土産などの情報をご案内。京都が楽しくなる特集記事やインタビューも。',
 '本音で語る京都観光案内。おすすめの散策コース、地図、お土産、珍しいお守り、食事からやカフェ案内まで、穴場も定番もいい所いい場所を紹介します。',
 'JR東海の京都観光情報【そうだ 京都、行こう。公式サイト】。京都旅行の便利帳、散策コース紹介ページ。京都の風景、お寺や神社、桜、紅葉、イベント、お祭り、お食事やお土産、宿泊などの情報をご案内。京都が楽しくなる特集 ...',
 'e京都ねっとは、京都人のスタッフによる京都の観光名所やイベント情報満載のポータルサイトです!お土産・グルメ情報・宿泊施設もご紹介しています!',
 '多数のアイテムから選べるセレクトクーポンで伝統の京料理、おばんざいや京漬物を堪能したり、タクシーやレンタカーで京都観光したり、京の文化体験もできちゃう！3,000円分のお土産券としても使えるよ！自由にチョイスして ...']

この検索の結果は以下のようになった。

1. JR東海の京都観光情報【そうだ 京都、行こう。公式サイト】。京都の風景、お寺や神社、イベント、お祭り、お食事やお土産などの情報をご案内。京都が楽しくなる特集記事やインタビューも。
2. 本音で語る京都観光案内。おすすめの散策コース、地図、お土産、珍しいお守り、食事からやカフェ案内まで、穴場も定番もいい所いい場所を紹介します。
3. JR東海の京都観光情報【そうだ 京都、行こう。公式サイト】。京都旅行の便利帳、散策コース紹介ページ。京都の風景、お寺や神社、桜、紅葉、イベント、お祭り、お食事やお土産、宿泊などの情報をご案内。京都が楽しくなる特集 ...
4. e京都ねっとは、京都人のスタッフによる京都の観光名所やイベント情報満載のポータルサイトです!お土産・グルメ情報・宿泊施設もご紹介しています!
5. 多数のアイテムから選べるセレクトクーポンで伝統の京料理、おばんざいや京漬物を堪能したり、タクシーやレンタカーで京都観光したり、京の文化体験もできちゃう！3,000円分のお土産券としても使えるよ！自由にチョイスして ...

検索結果より、クエリに含まれる文字列が含まれる文書が上位に現れていることがわかった。
一つ目や二つ目の検索結果では、クエリに含まれる文字列を同じくらい含んでいても、クエリに含まれていないような地名を多く含むほど順位が低くなっている。これは各地名が重要視されており重みが大きくなってしまうので、クエリとの類似度がさがってしまうからではないかと考えられる。
また三つ目の検索結果について、"京都"の他に"土産"のみを含んで"食事"を含まないものが上位に現れている。これは、

In [123]:
dictionary.token2id["土産"]

89

In [124]:
dictionary.token2id["食事"]

100

となっており"土産"が"食事"よりも出現回数が少ないため重要ととらえられており、このようなランキング結果になっていると考えられる。

## 任意課題（a） Okapi BM25
(１）に対して， Okapi BM25 に基づくランキングを行った。ここではgensim.summarization.bm25のget_bm25_weightsというメソッドを用いた。

### 実装したプログラム


In [157]:
# 文書の読み込み
# 1.のコーパスはjson形式で保管されている．
import json
with open("../data/kyoto_results_100.json", "r") as f:
    docs = json.load(f)
corpus = [d["bow"].lower().split() for d in docs] 
    
#　文書集合から必要な値を計算する
bm25 = gensim.summarization.bm25.BM25(corpus)
average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())

# クエリを受け取ってランキング結果を返す関数
# param query : クエリに含まれる単語の集合
def ranking_by_bm25(query):
    result = list(map(lambda i: (bm25.get_score(query, i, average_idf), i), range(len(docs))))
    result.sort()
    result.reverse()
    return result

# クエリを受け取ってランキング結果上位5位のsummaryを出力する関数
# param query : クエリに含まれる単語の集合
def getTop5Summary_by_bm25(query):
    result = ranking_by_bm25(query)
    return list(map(lambda i: docs[result[i][1]]["summary"], range(5)))

### (1)との比較
含まれる文書の数が多い"京都"と少ない"嵐山"でそれぞれ2通り検索し結果を比較した。

#### "京都"で検索した場合

In [158]:
# ベクトル空間モデルに基づいた検索
getTop5Summary({"京都"})

['京都の観光スポット情報、イベント・行事スケジュール、おすすめ観光コースのご紹介、全て無料で使える京都の写真ギャラリーを中心に、さまざまな京都観光の情報が満載の、京都観光情報ポータルサイトです。観光スポットの情報 ...',
 '奥深い京都の魅力をお伝えするため，数多くの京都観光情報をお伝えしています。イベント情報や花だより，施設情報など，京都観光に役立つ情報満載です。',
 'もっと京都が好きになる！京都通もおすすめする京都観光完全ガイド。京都の魅力がたっぷり詰まったグルメ・神社・寺院・和菓子・歴史ガイドをあますことなくランキングで紹介します！',
 '京都観光を楽しむ まとめ情報。京都の世界遺産や名所を紹介しています。 ... 京都観光 紅葉 京都紅葉の名称 京都紅葉おすすめ鑑賞スポット 東山や嵐山など、京都の紅葉は名所が広範囲にたくさんあるので',
 'JR東海の京都観光情報【そうだ 京都、行こう。公式サイト】。京都旅行の便利帳、散策コース紹介ページ。京都の風景、お寺や神社、桜、紅葉、イベント、お祭り、お食事やお土産、宿泊などの情報をご案内。京都が楽しくなる特集 ...']

これを実行すると、以下のようなランキング上位5つが得られた。

1. 京都の観光スポット情報、イベント・行事スケジュール、おすすめ観光コースのご紹介、全て無料で使える京都の写真ギャラリーを中心に、さまざまな京都観光の情報が満載の、京都観光情報ポータルサイトです。観光スポットの情報 ...
2. 奥深い京都の魅力をお伝えするため，数多くの京都観光情報をお伝えしています。イベント情報や花だより，施設情報など，京都観光に役立つ情報満載です。
3. もっと京都が好きになる！京都通もおすすめする京都観光完全ガイド。京都の魅力がたっぷり詰まったグルメ・神社・寺院・和菓子・歴史ガイドをあますことなくランキングで紹介します！
4. 京都観光を楽しむ まとめ情報。京都の世界遺産や名所を紹介しています。 ... 京都観光 紅葉 京都紅葉の名称 京都紅葉おすすめ鑑賞スポット 東山や嵐山など、京都の紅葉は名所が広範囲にたくさんあるので
5. JR東海の京都観光情報【そうだ 京都、行こう。公式サイト】。京都旅行の便利帳、散策コース紹介ページ。京都の風景、お寺や神社、桜、紅葉、イベント、お祭り、お食事やお土産、宿泊などの情報をご案内。京都が楽しくなる特集 ...

In [159]:
# bm25を用いた検索
getTop5Summary_by_bm25({"京都"})

['京言葉 京の花街 京町家 京料理 京都市内の通り 京都の元学区 京都の難読地名 小京都 そうだ 京都、行こう。 京都・東山花灯路 京都・嵐山花灯路 京阪神 京阪 光源氏、源氏物語 外部リンク 京都 歴史文化データリンク by KYOTO DOTCOM',
 '京都観光を楽しむ まとめ情報。京都の世界遺産や名所を紹介しています。 ... 京都観光 紅葉 京都紅葉の名称 京都紅葉おすすめ鑑賞スポット 東山や嵐山など、京都の紅葉は名所が広範囲にたくさんあるので',
 '京都観光案内サイト。おいでやす京都! 京都観光/旅行・ぶらり伏見では、ぶらり京都伏見を観光気分で散策しながら、京都の観光名所・観光スポットを紹介致します。（京都・観光文化検定試験/伏見編もチャレンジ出来ます。',
 'もっと京都が好きになる！京都通もおすすめする京都観光完全ガイド。京都の魅力がたっぷり詰まったグルメ・神社・寺院・和菓子・歴史ガイドをあますことなくランキングで紹介します！',
 '京都の観光スポット情報、イベント・行事スケジュール、おすすめ観光コースのご紹介、全て無料で使える京都の写真ギャラリーを中心に、さまざまな京都観光の情報が満載の、京都観光情報ポータルサイトです。観光スポットの情報 ...']

これを実行すると、以下のようなランキング上位5つが得られた。

1. 京言葉 京の花街 京町家 京料理 京都市内の通り 京都の元学区 京都の難読地名 小京都 そうだ 京都、行こう。 京都・東山花灯路 京都・嵐山花灯路 京阪神 京阪 光源氏、源氏物語 外部リンク 京都 歴史文化データリンク by KYOTO DOTCOM
2. 京都観光を楽しむ まとめ情報。京都の世界遺産や名所を紹介しています。 ... 京都観光 紅葉 京都紅葉の名称 京都紅葉おすすめ鑑賞スポット 東山や嵐山など、京都の紅葉は名所が広範囲にたくさんあるので
3. 京都観光案内サイト。おいでやす京都! 京都観光/旅行・ぶらり伏見では、ぶらり京都伏見を観光気分で散策しながら、京都の観光名所・観光スポットを紹介致します。（京都・観光文化検定試験/伏見編もチャレンジ出来ます。
4. もっと京都が好きになる！京都通もおすすめする京都観光完全ガイド。京都の魅力がたっぷり詰まったグルメ・神社・寺院・和菓子・歴史ガイドをあますことなくランキングで紹介します！
5. 京都の観光スポット情報、イベント・行事スケジュール、おすすめ観光コースのご紹介、全て無料で使える京都の写真ギャラリーを中心に、さまざまな京都観光の情報が満載の、京都観光情報ポータルサイトです。観光スポットの情報 ...

#### "嵐山"で検索した場合

In [160]:
# ベクトル空間モデルに基づいた検索
getTop5Summary({"嵐山"})

['京都嵐山観光おすすめ！人気散策コースから周辺グルメまでご紹介！観光 京都の代表的な観光地で桜や紅葉の名所にもなっている嵐山。一日過ごしても飽きないくらい観たり食べたり買い物したりする人気スポットがいっぱいです。',
 '嵐山といえば、京都においても屈指のおすすめ観光スポットですね。今回は嵐山に注目してこれだけは外せない！な、人気の名所や定番の観光スポットをランキング形式でご案内します！デートでも家族旅行でも間違いなしの ...',
 '春は桜、秋は紅葉が全山を覆う嵐山。この嵐山を起点として嵯峨野の小径を散策するコースは、歴史や文学に関わる史跡が豊富にあります。家族旅行からカップル、一人旅まで、それぞれに楽しめる幅と奥行きがある散策コースです。',
 '見所の多い京都観光の中でも、特に人気が高い嵐山・嵯峨野エリア。この辺りは、古くから貴族や文人たちが都を逃れ「わび住まい」をした風流の地だけあり、周囲の山々が紅葉に染まる晩秋ともなれば、その風情もひとしおです。',
 '京都観光を楽しむ まとめ情報。京都の世界遺産や名所を紹介しています。 ... 京都観光 紅葉 京都紅葉の名称 京都紅葉おすすめ鑑賞スポット 東山や嵐山など、京都の紅葉は名所が広範囲にたくさんあるので']

これを実行すると、以下のようなランキング上位5つが得られた。

1. 京都嵐山観光おすすめ！人気散策コースから周辺グルメまでご紹介！観光 京都の代表的な観光地で桜や紅葉の名所にもなっている嵐山。一日過ごしても飽きないくらい観たり食べたり買い物したりする人気スポットがいっぱいです。
2. 嵐山といえば、京都においても屈指のおすすめ観光スポットですね。今回は嵐山に注目してこれだけは外せない！な、人気の名所や定番の観光スポットをランキング形式でご案内します！デートでも家族旅行でも間違いなしの ...
3. 春は桜、秋は紅葉が全山を覆う嵐山。この嵐山を起点として嵯峨野の小径を散策するコースは、歴史や文学に関わる史跡が豊富にあります。家族旅行からカップル、一人旅まで、それぞれに楽しめる幅と奥行きがある散策コースです。
4. 見所の多い京都観光の中でも、特に人気が高い嵐山・嵯峨野エリア。この辺りは、古くから貴族や文人たちが都を逃れ「わび住まい」をした風流の地だけあり、周囲の山々が紅葉に染まる晩秋ともなれば、その風情もひとしおです。
5. 京都観光を楽しむ まとめ情報。京都の世界遺産や名所を紹介しています。 ... 京都観光 紅葉 京都紅葉の名称 京都紅葉おすすめ鑑賞スポット 東山や嵐山など、京都の紅葉は名所が広範囲にたくさんあるので

In [161]:
# bm25を用いた検索
getTop5Summary_by_bm25({"嵐山"})

['嵐山といえば、京都においても屈指のおすすめ観光スポットですね。今回は嵐山に注目してこれだけは外せない！な、人気の名所や定番の観光スポットをランキング形式でご案内します！デートでも家族旅行でも間違いなしの ...',
 '春は桜、秋は紅葉が全山を覆う嵐山。この嵐山を起点として嵯峨野の小径を散策するコースは、歴史や文学に関わる史跡が豊富にあります。家族旅行からカップル、一人旅まで、それぞれに楽しめる幅と奥行きがある散策コースです。',
 '京都嵐山観光おすすめ！人気散策コースから周辺グルメまでご紹介！観光 京都の代表的な観光地で桜や紅葉の名所にもなっている嵐山。一日過ごしても飽きないくらい観たり食べたり買い物したりする人気スポットがいっぱいです。',
 '見所の多い京都観光の中でも、特に人気が高い嵐山・嵯峨野エリア。この辺りは、古くから貴族や文人たちが都を逃れ「わび住まい」をした風流の地だけあり、周囲の山々が紅葉に染まる晩秋ともなれば、その風情もひとしおです。',
 '金閣寺は京都を代表する観光地のひとつ。銀閣寺・嵐山・清水寺などとならび、五本の指に入る人気スポットです。ところが京都観光は移動が大変。交通手段も複雑です。そこで今回は、金閣寺から各観光名所・人気スポットへの ...']

これを実行すると、以下のようなランキング上位5つが得られた。

1. 嵐山といえば、京都においても屈指のおすすめ観光スポットですね。今回は嵐山に注目してこれだけは外せない！な、人気の名所や定番の観光スポットをランキング形式でご案内します！デートでも家族旅行でも間違いなしの ...
2. 春は桜、秋は紅葉が全山を覆う嵐山。この嵐山を起点として嵯峨野の小径を散策するコースは、歴史や文学に関わる史跡が豊富にあります。家族旅行からカップル、一人旅まで、それぞれに楽しめる幅と奥行きがある散策コースです。
3. 京都嵐山観光おすすめ！人気散策コースから周辺グルメまでご紹介！観光 京都の代表的な観光地で桜や紅葉の名所にもなっている嵐山。一日過ごしても飽きないくらい観たり食べたり買い物したりする人気スポットがいっぱいです。
4. 見所の多い京都観光の中でも、特に人気が高い嵐山・嵯峨野エリア。この辺りは、古くから貴族や文人たちが都を逃れ「わび住まい」をした風流の地だけあり、周囲の山々が紅葉に染まる晩秋ともなれば、その風情もひとしおです。
5. 金閣寺は京都を代表する観光地のひとつ。銀閣寺・嵐山・清水寺などとならび、五本の指に入る人気スポットです。ところが京都観光は移動が大変。交通手段も複雑です。そこで今回は、金閣寺から各観光名所・人気スポットへの ...

以上の結果より、含まれる文書の数が多い"京都"では、手法により結果が大きく変化したことがわかった。　特に、bm25では"京都"という言葉が多く含まれる文書ほど上位に現れている。また含まれる文書の数が少ない"嵐山"では上位5つに現れる文書はそれほど変わりないが、順位が変化することがわかった。